In [1]:
# Task 1: Import libraries
import pennylane as qml
from pennylane import numpy as np

Create a function that generates the Quantum Fourier Transform of a number $m$ via a quantum circuit with $n$-qubits
- Arguments:
  - `n`: The number of qubits in the variational quantum circuit
  - `m`: The number used to train the variational quantum circuit. Its values ranges from $0$ to $2^n - 1$
  - `steps`: The number of iterations during the optimization process
  - `learning_rate`: The step size of the optimizer; its default value should be set to 0.01
- Return values:
  - `circuit`: The variational quantum circuit comprizing $n$ qubits
  - `param_arr`: Consists of the parameter values after each optimization step
    - Shape = `(steps, n)`
  - `cost_arr`: Consists of the cost value of the circuit after each optimization step
    - Size = `n`
  - `cost`: The cost function

In [2]:
def qft_via_qml(n: int, m: int, steps: int, learning_rate = 0.01):
    # Task 2: Load a Quantum Device
    
    # Task 3-6: Create the Quantum Circuit
    
    # Task 7: Create the Cost Function
    
    # Task 8: Initialize the Optimizer
    
    # Task 9: Construct the Optimization Block
    
    return circuit, param_arr, cost_arr, cost

In [ ]:
# Task 10: Test the Optimizer


In [ ]:
# Task 11: Visualize the Optimization Process